In [19]:
import boto3, json, io, os
import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.aws_connectors as aws
from datetime import datetime
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# instanciate AWS connection
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket_name = 'calfdata'
AWS_connection = aws.aws_instantiate(instance_aws, bucket_name)
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)


In [21]:
#Iterate on all ETS folders
columns_=['Siren','Type','Adresse_Ligne1','Adresse_Ligne2','Adresse_Ligne3',
          'Code_Postal','Ville','Code_Commune','ID_Etablissement','Date_Greffe','Libelle_Evt']
columns_add = columns_ + [ 'file_timestamp', 'Libelle_Evt2']
df = pd.DataFrame(columns=columns_add)

In [ ]:
log = []
error_log = []

for y in range(2018, 2020):
    for m in range(1, 13):
        m_ = '0'+ str(m) if m<10 else str(m)
        prefix_='{}/{}/{}/{}'.format('INPI/TC_1/Flux',y,m_,'ETS/EVT')
        df = pd.DataFrame(columns=columns_add)
        print('starting ' + prefix_)
        c=0
        e=0
        for obj in list(bucket.objects.filter(Prefix=prefix_)):
            key = obj.key
            body = obj.get()['Body'].read()
            try:

                df_ = pd.read_csv(io.BytesIO(body), header=0, dtype=str, sep = ';',usecols=columns_,error_bad_lines=False)
                
                #Adding information from filename
                s=key.split('/')
                filename=s[len(s)-1]
                #print(filename)
                f_s=filename.split('_')

                # Add timestamp
                datetime_str = '{}_{}'.format(f_s[2],f_s[3])
                datetime_object = datetime.strptime(datetime_str, '%Y%m%d_%H%M%S')
                #print(datetime_object)
                df_['file_timestamp']=datetime_object

                # Add detailed Event Label
                df_['Libelle_Evt2']='_'.join(f_s[6:-1])
                
                df = df.append(df_,sort=False)
                c+=1
            except:
                print('Error on file:' + key)
                error_log.append({'key': key})
                e+=1
            pass

        print('End of ' + prefix_ + ' : ' + str(len(df)) + ' rows now loaded.')
        log.append({
            'path': prefix_,
            'files': c,
            'rows': len(df),
            'errors': e
        })
        
        df.to_csv(r'{}/{}{}.{}'.format('ets_enriched',y,m_, 'csv'))#'gz'),compression='gzip')
        json_name = '{}/{}{}-{}'.format('logs',y,m_, 'log.json')
        #print('json_name ' + json_name)
        json_errorname = '{}/{}{}-{}'.format('logs',y,m_, 'errorlog.json')
        with open(json_name, 'w') as outfile:
            json.dump(log, outfile)
        with open(json_errorname, 'w') as outfile:
            json.dump(error_log, outfile)
        #df = []

In [ ]:
filename='0101_10_20170519_063335_10_ets_supprime_EVT.csv'
f_s=filename.split('_')

# Add timestamp
datetime_str = '{}_{}'.format(f_s[2],f_s[3])
datetime_object = datetime.strptime(datetime_str, '%Y%m%d_%H%M%S')
datetime_object.second

In [ ]:
#'INPI/TC_1/Flux/2019/05/ETS/EVT/3302_34_20170530_082023_9_ets_nouveau_modifie_EVT.csv'

In [ ]:
# RAF : 
# save and remove logs
#os.remove(json_name)
#os.remove(json_errorname)
# Read : https://dluo.me/s3databoto3
# Read : https://medium.com/@victor.perez.berruezo/download-a-csv-file-from-s3-and-create-a-pandas-dataframe-in-python-ffdb08c2967c

In [ ]:
#Errors logged
"INPI/TC_1/Flux/2017/05/ETS/EVT/5002_10_20170519_071930_9_ets_nouveau_modifie_EVT.csv"
"INPI/TC_1/Flux/2017/08/ETS/EVT/0702_70_20170819_080206_9_ets_nouveau_modifie_EVT.csv"
"INPI/TC_1/Flux/2018/07/ETS/EVT/0101_310_20180705_065824_9_ets_nouveau_modifie_EVT.csv"
"INPI/TC_1/Flux/2018/10/ETS/EVT/7608_418_20181006_083003_10_ets_supprime_EVT.csv"

In [ ]:
df_ = pd.read_csv('ets_enriched/201705.csv')

In [ ]:
df_.shape

In [ ]:
df_['file_timestamp']=pd.to_datetime(df_['file_timestamp'])

In [ ]:
df_['year'] = df_['file_timestamp'].dt.year
df_['month'] = df_['file_timestamp'].dt.month

In [ ]:
df__2017_05 = df_.loc[lambda x: ( x['year'].isin(['2017'])) & 
                                               (x['month'].isin(['5']))]
df__2017_05.shape

In [ ]:
df__2017_05.drop_duplicates().shape